In [201]:
import xlwings as xw
import pandas as pd
import timestring
import numpy as np

In [202]:
import xlwings as xw
import pandas as pd
import timestring
import numpy as np

from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.utils import shuffle
from plotly.subplots import make_subplots
import plotly.graph_objects as go

import matplotlib.ticker as ticker

from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_absolute_error

from sklearn.metrics import mean_squared_error

from sklearn.model_selection import KFold

import matplotlib.pyplot as plt

### Загрузка и обработка данных по ценам и позициям

In [203]:
# загружаем данные по ценам и объему
data_rts = pd.read_csv('C:/Users/nicol/Python/Рынок/Эксперимент ФР/Эксперименты excel/SPFB.RTS.txt', sep=',', index_col=2)

# меняем формат столбца и делаем в качестве индекса
data_rts.index = pd.to_datetime(data_rts.index, format="%Y%m%d")
#data_rts.tail()

# создадим столбец со средней ценой в минуту
data_rts['price_mean'] = data_rts['<CLOSE>']
#data_rts.head()

# отсортируем по дате и возьмем среденее по цене за день
data_rts_mean = data_rts.pivot_table(index=['<DATE>'], values = 'price_mean', aggfunc=('mean'))
#data_rts_mean.head()

# открываем и читаем книгу по данным ФЗ ЮЛ
wb = xw.Book('C:/Users/nicol/Python/Рынок/Эксперимент ФР/Эксперименты excel/Юр-Физ лица цена2.xlsx') # Открываем книгу
data_excel = wb.sheets['RTS-12.20 сокр'] # Читаем лист Данные

# считываем диапазон
data_pd = data_excel.range('A1:F1117').options(pd.DataFrame, index = False).value

# меняем формат столбца 
data_pd['Дата'] = pd.to_datetime(data_pd['Дата'])

# устанавливаем в качестве индекса дату и удаляем лишний столбцец
data_pd.index = data_pd['Дата']
data_pd = data_pd.drop(['Дата'], axis = 1)

# загружаем данные по ценам и объему
data_rts_finam = pd.read_csv('C:/Users/nicol/Python/Рынок/Эксперимент ФР/Эксперименты excel/SPFB.RTS-9.21.csv', sep=',', index_col=2)

# меняем формат столбца и делаем в качестве индекса
data_rts_finam.index = pd.to_datetime(data_rts_finam.index, format="%Y%m%d")

data_rts_finam['price_mean'] = data_rts_finam['<CLOSE>'] 

data_rts_finam_2 = data_rts_finam.drop(['<TICKER>','<PER>','<TIME>','<OPEN>','<HIGH>','<LOW>','<CLOSE>','<VOL>'], axis = 1)

data_rts_sum = pd.concat([data_rts_mean,data_rts_finam_2], axis = 0)

# выбираем из первой таблицы актив и соединяем таблицы с ценами
data_pd_n = data_pd.join(data_rts_sum, how = 'left')

# удаляем пробелы в цифрах
data_pd_n = data_pd_n.replace(r'\s+','',regex=True)

# переводим в числовой тип
data_pd_n = data_pd_n.apply(pd.to_numeric, errors='coerce')

# удаляем столбцы с низким значением позиций юрлиц
data_pd_n = data_pd_n.loc[data_pd_n['ЮЛДП'] > 1000]

# удаляем строки с нулевыми значениями
#data_pd_n = data_pd_n[pd.notnull(data_pd_n['price_mean'])]

data_pd_rts = data_pd_n

# добавим в % позиционирование ФЗ ЮЛ
data_pd_rts['ФЗДП%'] = data_pd_rts['ФЗДП']/(data_pd_rts['ФЗДП'] + data_pd_rts['ФЗКП'])
data_pd_rts['ЮЛКП%'] = data_pd_rts['ЮЛКП']/(data_pd_rts['ЮЛДП'] + data_pd_rts['ЮЛКП'])
data_pd_rts['ЮЛДП%'] = data_pd_rts['ЮЛДП']/(data_pd_rts['ЮЛДП'] + data_pd_rts['ЮЛКП'])
data_pd_rts['ФЗ_ЮЛ'] =(data_pd_rts['ФЗДП'] + data_pd_rts['ФЗКП'])/(data_pd_rts['ЮЛДП'] + data_pd_rts['ЮЛКП'])

data_pd_rts.sort_index(inplace=True, ascending=True)

data_pd_rts.to_csv('data_pd_rts.csv')

#### Даты эксперации квартального фьючерса 

In [204]:
data_experac = wb.sheets['Дата экспер'] # Читаем лист Данные

# считываем диапазон
experac = data_experac.range('C5:C17').options(pd.Series, index = False).value

# меняем формат столбца 
experac = pd.to_datetime(experac)

#### Даты эксперации месячных опционов

In [205]:
data_experac_opc = wb.sheets['Дата экспер'] # Читаем лист Данные

# считываем диапазон
experac_opc = data_experac_opc.range('E5:E41').options(pd.Series, index = False).value

# меняем формат столбца 
experac_opc = pd.to_datetime(experac_opc)

#### Создание графика ФЗ_ЮЛ

In [206]:
def train(window1, window2, window3):  
    '''
    Фичи: 
    1. window1 - скользящее среднее для цены
    2. window2 - скользящее среднее для ЮЛДП%_ФЗДП% 
    3. window3 - скользящее среднее для ФЗ_ЮЛ

    '''
    train = pd.DataFrame()

    # добавим СКСР для цены window1
    sksr_price = data_pd_rts['price_mean'].shift().rolling(window=window1).median()
    
    # добавим СКСР для ЮЛДП%_ФЗДП% window2
    sksr_ЮЛДП_ФЗДП = pd.DataFrame()
    sksr_ЮЛДП_ФЗДП['ewmaФЗДП%'] = data_pd_rts['ФЗДП%'].shift().rolling(window=window2).median()
    sksr_ЮЛДП_ФЗДП['ewmaЮЛДП%'] = data_pd_rts['ЮЛДП%'].shift().rolling(window=window2).median()
    sksr_ЮЛДП_ФЗДП['ЮЛДП%_ФЗДП%_sksr'] = sksr_ЮЛДП_ФЗДП['ewmaЮЛДП%']/sksr_ЮЛДП_ФЗДП['ewmaФЗДП%']
    
    # ЮЛДП%_ФЗДП% обычная не СКСР
    train['ЮЛДП%_ФЗДП%'] = data_pd_rts['ЮЛДП%']/data_pd_rts['ФЗДП%']
     
    # добавим СКСР объема позиций ФЗ к ЮЛ window3
    sksr_ФЗ_ЮЛ = data_pd_rts['ФЗ_ЮЛ'].shift().rolling(window=window3).median()
    
    # вставим все столбцы в train
    train['price_sksr'] = sksr_price
    train['ЮЛДП%_ФЗДП%_sksr'] = sksr_ЮЛДП_ФЗДП['ЮЛДП%_ФЗДП%_sksr']
    train['ФЗ_ЮЛ_sksr'] = sksr_ФЗ_ЮЛ
    train['ewmaФЗДП%'] = sksr_ЮЛДП_ФЗДП['ewmaФЗДП%']
    train['ewmaЮЛДП%'] = sksr_ЮЛДП_ФЗДП['ewmaЮЛДП%']
    train['price_mean'] = data_pd_rts['price_mean']
    
    #добавим объемы по отдельности
    
    train['ФЗ'] = data_pd_rts['ФЗДП'] + data_pd_rts['ФЗКП']
    train['ЮЛ'] = data_pd_rts['ЮЛДП'] + data_pd_rts['ЮЛКП']
    
    # ФЗДП и ЮЛДП без скср
    train['ФЗДП%'] = data_pd_rts['ФЗДП%']
    train['ЮЛДП%'] = data_pd_rts['ЮЛДП%']
    train['ФЗ_ЮЛ'] = data_pd_rts['ФЗ_ЮЛ']
    

    #train = train.dropna().reset_index()
    #train = train.drop(['index'], axis =1)

    return train

In [207]:
# данные для построения графика ФЗ_ЮЛ
z = 1
opit_1 = train(z,z,z)

In [208]:
#opit_1.head()

### Создание осциллятора

In [209]:
def data_new(data_pd_rts, day, window1, window2, window3, k1, lag1, lag2):  
    
    '''
    Фичи (day, window1, window2, window3, k1, lag1, lag2): 
    
    1. day - изменение цены за day дней вперед (будущее)
    2. window1 - СКСР price_mean
    3. window3 - СКСР объема позиций ФЗ к ЮЛ
    4. window2 - СКСР ЮЛДП%_ФЗДП% 
    5. k1 - изменение за k1 дней (отдельные столбцы) 
    6. lag1 - лаг по значениям 'ЮЛДП%_ФЗДП%_otnosit' (отдельные столбцы) 
    7. lag2 - лаг по значениям 'ФЗ_ЮЛ_otnosit' (отдельные столбцы) 
       
    Цель:
    изменение цены вперед day дней

    '''
    train = pd.DataFrame()

    # расчитываем изменение цены за day дней
    change_rts = data_pd_rts['price_mean'].pct_change(periods=day)
       
    # добавим СКСР для цены window1
    sksr_price = data_pd_rts['price_mean'].shift().rolling(window=window1).median()
    train['price_sksr'] = sksr_price
    train['price_mean'] = data_pd_rts['price_mean']
    train['price_sksr_otnosit'] = (train['price_mean'] - train['price_sksr'])/train['price_sksr']
   
    # добавим изменение цены за дней k1 в отдельных столбцах
    for price_izm in range(1, k1 + 1):
        train['price_izm_{}'.format(price_izm)] = data_pd_rts['price_mean'].pct_change(periods=price_izm)
    
    #price_izm = data_pd_rts['price_mean'].pct_change(periods=k1)
   
    # добавим СКСР для ЮЛДП%_ФЗДП% window2
    sksr_ЮЛДП_ФЗДП = pd.DataFrame()
    sksr_ЮЛДП_ФЗДП['ewmaФЗДП%'] = data_pd_rts['ФЗДП%'].shift().rolling(window=window2).median()
    sksr_ЮЛДП_ФЗДП['ewmaЮЛДП%'] = data_pd_rts['ЮЛДП%'].shift().rolling(window=window2).median()
    
    sksr_ЮЛДП_ФЗДП['ЮЛДП%_ФЗДП%_sksr'] = sksr_ЮЛДП_ФЗДП['ewmaЮЛДП%']/sksr_ЮЛДП_ФЗДП['ewmaФЗДП%']
    
    # отношение ЮЛДП%_ФЗДП% без скср
    sksr_ЮЛДП_ФЗДП['ЮЛДП%_ФЗДП%'] = data_pd_rts['ЮЛДП%']/data_pd_rts['ФЗДП%']  
    
    # добавим СКСР объема позиций ФЗ к ЮЛ window3
    sksr_ФЗ_ЮЛ = data_pd_rts['ФЗ_ЮЛ'].shift().rolling(window=window3).median()

    # вставим все столбцы в train
    train['price_change'] = change_rts.shift(-day)
    train['ЮЛДП%_ФЗДП%_sksr'] = sksr_ЮЛДП_ФЗДП['ЮЛДП%_ФЗДП%_sksr']
    train['ЮЛДП%_ФЗДП%'] = sksr_ЮЛДП_ФЗДП['ЮЛДП%_ФЗДП%']
    
    # отношение ЮЛДП%_ФЗДП% к ЮЛДП%_ФЗДП% скср
    train['ЮЛДП%_ФЗДП%_otnosit'] = (train['ЮЛДП%_ФЗДП%']-train['ЮЛДП%_ФЗДП%_sksr'])/train['ЮЛДП%_ФЗДП%_sksr']
        
    # добавим лаг значений  'ЮЛДП%_ФЗДП%_otnosit' в отдельных столбцах
    for lag_1 in range(1, lag1 + 1):
        
        train['ЮЛДП%_ФЗДП%_{}'.format(lag_1)] = train['ЮЛДП%_ФЗДП%'].pct_change(lag_1)
    
    train['ФЗ_ЮЛ'] = data_pd_rts['ФЗ_ЮЛ']

    train['ФЗ_ЮЛ_sksr'] = sksr_ФЗ_ЮЛ
    
    # отношение ФЗ_ЮЛ к ФЗ_ЮЛ скср
    train['ФЗ_ЮЛ_otnosit'] = (train['ФЗ_ЮЛ']-train['ФЗ_ЮЛ_sksr'])/train['ФЗ_ЮЛ_sksr']
    
    # добавим лаг значений  'ЮЛДП%_ФЗДП%_otnosit' в отдельных столбцах
    for lag_2 in range(1, lag2 + 1):
        train['ФЗ_ЮЛ_{}'.format(lag_2)] = train['ФЗ_ЮЛ'].pct_change(lag_2)

    train['ewmaФЗДП%'] = sksr_ЮЛДП_ФЗДП['ewmaФЗДП%']
    train['ewmaЮЛДП%'] = sksr_ЮЛДП_ФЗДП['ewmaЮЛДП%']

    #train = train.dropna().reset_index()
    #train = train.drop(['index'], axis =1)    

    #train = train.dropna().reset_index()
    #train = train.drop(['ЮЛДП%_ФЗДП%_sksr', 'ЮЛДП%_ФЗДП%', 'ЮЛДП%_ФЗДП%_izm', 'ФЗ_ЮЛ', 'ФЗ_ЮЛ_sksr', 'ewmaФЗДП%', 'ewmaЮЛДП%', 'price_mean'], axis =1)

    #train = train.dropna()
        
    return train

#### Экспорт осциллятора в отдельный файл

In [210]:
opit = data_new(data_pd_rts, 1, 21, 36, 156, 6, 3, 2) 
opit.to_csv('opit_rts.csv')

In [211]:
# читаем данные по si для добавления на график
opit_si = pd.read_csv('C:/Users/nicol/Python/Рынок/Эксперимент ФР/opit_si.csv', sep=',', index_col=0)

fzul_si = pd.read_csv('C:/Users/nicol/Python/Рынок/Эксперимент ФР/fzul_si.csv', sep=',', index_col=0)

### График ФЗ_ЮЛ

In [212]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
go.Scatter(x=data_pd_rts.index, y=opit_1['price_mean'], 
            name = 'price_mean',  line=dict(color='red', width=3)),
            secondary_y=True)

fig.add_trace(
go.Scatter(x=data_pd_rts.index, y=opit_1['price_sksr'], 
            name = 'price_sksr',  line=dict(color='pink', width=2)),
            secondary_y=True)

fig.add_trace(
go.Scatter(x=data_pd_rts.index, y=opit_1['ФЗДП%'], 
            name = 'ewmaФЗДП%',  line=dict(color='green', width=2)),
            secondary_y=False)

fig.add_trace(
go.Scatter(x=data_pd_rts.index, y=opit_1['ЮЛДП%'], 
            name = 'ewmaЮЛДП%',  line=dict(color='brown', width=2)),
            secondary_y=False)

fig.add_trace(
go.Scatter(x=data_pd_rts.index, y=opit_1['ФЗ_ЮЛ'],
            name = 'ФЗ_ЮЛ', line=dict(color='yellow', width=3)),
            secondary_y=False)

fig.update_yaxes(title_text="отн.знач.", tick0 = 0.1, dtick = 0.1, secondary_y=False)
fig.update_yaxes(title_text="Цена", secondary_y=True)

fig.update_layout(
 #title_text="Зависимость уровня ФЗДП, ЮЛКП и ФЗ_ЮЛ от цены закрытия",
 #title_font_size=15,
 #xaxis_rangeslider_visible=True, # слайдер,
 xaxis_title="Дата",height = 600,legend=dict(orientation="h", yanchor="bottom",y=1.02, xanchor="right", x=1 ))
       
fig.show()


# 2 график

fig2 = make_subplots(specs=[[{"secondary_y": True}]])

fig2.add_trace(
go.Scatter(x=data_pd_rts.index, y=opit_1['price_mean'], 
            name = 'price_mean',  line=dict(color='red', width=3)),
            secondary_y=True)

fig2.add_trace(
go.Scatter(x=data_pd_rts.index, y=opit_1['price_sksr'], 
            name = 'price_sksr',  line=dict(color='pink', width=3)),
            secondary_y=True)

fig2.add_trace(
go.Scatter(x=data_pd_rts.index, y=opit_1['ЮЛДП%_ФЗДП%'],
            name = 'ЮЛДП%_ФЗДП%', line=dict(color='purple', width=2)),
            secondary_y=False)

fig2.add_trace(
go.Scatter(x=data_pd_rts.index, y=opit_1['ФЗ_ЮЛ']*10,
            name = 'ФЗ_ЮЛ', line=dict(color='yellow', width=3)),
            secondary_y=False)

# fig2.add_trace(
# go.Scatter(x=data_pd_rts.index, y=opit_1['ФЗ']/10**4,
#             name = 'ФЗ', line=dict(color='green', width=3)),
#             secondary_y=False)

# fig2.add_trace(
# go.Scatter(x=data_pd_rts.index, y=opit_1['ЮЛ']/10**4,
#             name = 'ЮЛ', line=dict(color='brown', width=3)),
#             secondary_y=False)


fig2.update_yaxes(title_text="Цена", secondary_y=True)
fig2.update_yaxes(tick0 = 10000, dtick = 5000, secondary_y=True)

fig2.update_layout(
 xaxis_tickformat = "%d %B (%a)<br>%Y",xaxis_title="Дата",title="ФЗ_ЮЛ",height = 600,
 legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1 ))
       
fig2.show()

### График осциллятора

In [213]:
fig3 = make_subplots(specs=[[{"secondary_y": True}]])

fig3.add_trace(
go.Scatter(x=data_pd_rts.index, y=data_pd_rts['price_mean'], 
            name = 'price_mean',  line=dict(color='red', width=3)),
            secondary_y=True)

fig3.add_trace(
go.Scatter(x=data_pd_rts.index, y=opit['price_sksr'], 
            name = 'price_sksr',  line=dict(color='pink', width=3)),
            secondary_y=True)

fig3.add_trace(
go.Scatter(x=data_pd_rts.index, y=opit['ЮЛДП%_ФЗДП%_otnosit'],
            name = 'ЮЛДП%_ФЗДП%_otnosit', line=dict(color='purple', width=2)),
            secondary_y=False)

fig3.add_trace(
go.Scatter(x=data_pd_rts.index, y=opit['ФЗ_ЮЛ_otnosit'],
            name = 'ФЗ_ЮЛ_otnosit', line=dict(color='yellow', width=3)),
            secondary_y=False)

# график по сишке
fig3.add_trace(
go.Scatter(x=opit_si.index, y=opit_si['ЮЛДП%_ФЗДП%_otnosit_sksr'],
            name = 'Si_ЮЛДП%_ФЗДП%_otnosit_sksr', line=dict(color='green', width=2)),
            secondary_y=False)

fig3.add_trace(
go.Scatter(x=data_pd_rts.index, y=np.zeros((len(data_pd_rts.index))),
            line=dict(color='blue', width=1)),
            secondary_y=False)

# даты экспирации месячных опционов
for i in range(len(experac_opc)):
    oy = np.arange(70,180)
    ox = np.full((len(oy)),experac_opc[i])
    oxs = pd.Series(ox) 
    fig3.add_trace(
    go.Scatter(x=oxs, y=oy*1000,line=dict(color='lightblue', width=1)),secondary_y=True)

# даты экспирации фьючерсов
for i in range(len(experac)):
    oy = np.arange(70,180)
    ox = np.full((len(oy)),experac[i])
    oxs = pd.Series(ox) 
    fig3.add_trace(
    go.Scatter(x=oxs, y=oy*1000,line=dict(color='blue', width=1)),secondary_y=True)

fig3.update_yaxes(title_text="Цена", secondary_y=True)
fig3.update_yaxes(tick0 = 10000, dtick = 5000, secondary_y=True)

fig3.update_layout(xaxis_tickformat = "%d %B (%a)<br>%Y",xaxis_title="Дата",title="Осциллятор ФЗ_ЮЛ",height = 600)

fig3.update(layout_showlegend=False)  

fig3.show()

#### Некоторые дополнительные эксперименты

In [214]:
# from datetime import datetime

# date_string = "21 June, 2018" 
# #print("date_string =", date_string) 
# date_object = datetime.strptime(date_string, "%d %B, %Y") 
# print("date_object =", date_object)


Условие что цена больше скользящей средней

In [215]:
opit['price_bolshe_sksr'] = opit['price_mean'] > opit['price_sksr']

opit=opit.applymap(lambda x: int(x) if type(x)==bool else x)
opit = opit.dropna()

In [216]:
# opit[(opit['ЮЛДП%_ФЗДП%_otnosit'] > 0.1) & (opit['ФЗ_ЮЛ_otnosit'] > 0) & (opit['ФЗ_ЮЛ_otnosit'] < 0.2)]['price_bolshe_sksr'].hist()

In [217]:
c = opit[(opit['ЮЛДП%_ФЗДП%_otnosit'] > 0) & (opit['ФЗ_ЮЛ_otnosit'] > 0)]['price_bolshe_sksr'].sum()
d = opit[(opit['ЮЛДП%_ФЗДП%_otnosit'] > 0) & (opit['ФЗ_ЮЛ_otnosit'] > 0)]['price_bolshe_sksr'].count()
c/d

0.9372822299651568

In [218]:
#c/d

In [219]:
# opit[(opit['ЮЛДП%_ФЗДП%_otnosit'] > 0 ) & (opit['ФЗ_ЮЛ_otnosit'] > 0)]['price_bolshe_sksr'].hist()

In [220]:
c = opit[(opit['ЮЛДП%_ФЗДП%_otnosit'] < 0.1) & (opit['ФЗ_ЮЛ_otnosit'] < 0.1)]['price_bolshe_sksr'].sum()
d = opit[(opit['ЮЛДП%_ФЗДП%_otnosit'] < 0.1) & (opit['ФЗ_ЮЛ_otnosit'] < 0.1)]['price_bolshe_sksr'].count()
1-c/d

0.5809018567639257

In [221]:
# opit[(opit['ЮЛДП%_ФЗДП%_otnosit'] < 0 ) & (opit['ФЗ_ЮЛ_otnosit'] < 0)]['price_bolshe_sksr'].hist()

Короткие позиции

Подбор параметров windows 1, 2, 3 для более достоверных условий что цена больше скср

In [222]:
#od_1 = pd.DataFrame()
# n = 0
# for window1 in range(1, 207, 5):
#     for window2 in range(1, 207, 5):
#         for window3 in range(1, 207, 5):
            
#             n += 1
#             opit = data_new(data_pd_rts, 2, window1, window2, window3, 6, 3, 2)
#             opit['price_bolshe_sksr'] = opit['price_mean'] > opit['price_sksr']

#             opit=opit.applymap(lambda x: int(x) if type(x)==bool else x)
#             opit = opit.dropna()
            
#             c = opit[(opit['ЮЛДП%_ФЗДП%_otnosit'] > 0) & (opit['ФЗ_ЮЛ_otnosit'] > 0)]['price_bolshe_sksr'].sum()
#             d = opit[(opit['ЮЛДП%_ФЗДП%_otnosit'] > 0) & (opit['ФЗ_ЮЛ_otnosit'] > 0)]['price_bolshe_sksr'].count()
            
#             count = c/d
            
#             mod_1.loc[n, 'window1'] = window1
#             mod_1.loc[n, 'window2'] = window2
#             mod_1.loc[n, 'window3'] = window3
#             mod_1.loc[n, 'count'] = count

In [223]:
# mod_1.loc[mod_1['count'] == mod_1['count'].max()]

In [224]:
#mod_1.to_csv('осциллятор 10.04 шаг 5.csv')

In [225]:
#mod_1.loc[mod_1['count'] == mod_1['count'].max()]

In [226]:
# w = 21
# mod_1.loc[(mod_1['window1'] == w)].loc[mod_1.loc[(mod_1['window1'] == w)]['count'] == mod_1.loc[(mod_1['window1'] == w)]['count'].max()]

In [227]:
#mod_1.loc[mod_1['count'] == mod_1['count'].min()]

In [228]:
#100**3